ПРИМЕРЫ - Убираем переносы строк  форматируем ...

In [ ]:
import re
import textwrap

text = """Это пример

     
   текста с

многочисленными
переносами."""


# удаляем пробелы в начале и конце строки
cleaned = text.strip()
print(cleaned + "\n")

# удаляем лишние пробелы
normalized = ' '.join(text.split())
print(normalized + "\n")

# убрать все повторяющиеся пробелы и переносы строк
normalized_all = re.sub(r'\s+', ' ', text).strip()
print(normalized_all + "\n")

# заменяем один или несколько \n на пробел
pressed = re.sub(r'\n+', ' ', text)
print(pressed + "\n")


# форматируем текст с шириной 30 символов
wrapped = textwrap.fill(text, width=30)
print(wrapped + "\n")

Сохраняет абзацы (каждый абзац — это блок текста, разделённый пустой строкой).
Убирает лишние пробелы внутри абзацев.
Убирает случайные пустые строки внутри абзацев.
Выводит каждый абзац с пустой строкой после.

In [ ]:
import re
import builtins

# Сохраняем оригинальный встроенный print
original_print = builtins.print

# Переопределяем print, чтобы добавлять пустую строку после
def print(*args, **kwargs):
    original_print(*args, **kwargs)
    original_print()  # пустая строка после

# Функция для очистки и форматирования текста в абзацы
def clean_paragraphs(text):
    # Разбиваем текст на абзацы (по двойным и более переносам строк)
    raw_paragraphs = re.split(r'\n\s*\n', text)

    cleaned_paragraphs = []
    for para in raw_paragraphs:
        # Убираем лишние пробелы и переносы внутри абзаца
        clean_para = re.sub(r'\s+', ' ', para).strip()
        if clean_para:  # исключаем пустые абзацы
            cleaned_paragraphs.append(clean_para)

    return cleaned_paragraphs


# Пример текста
raw_text = """
   Это   пример текста
с несколькими

   пробелами и переносами.

Вот второй абзац  с лишними   пробелами.
"""

paragraphs = clean_paragraphs(raw_text)

# Выводим каждый абзац с пустой строкой после
for p in paragraphs:
    print(p)

Склеиваем переносы через дефис

In [ ]:
import re

text = """при-вет как де-
ла? сегодня всё хо-
рошо"""

# склеиваем переносы через дефис
fixed_text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

print(fixed_text)

================================================

Нормализация текста с различными параметрами

In [ ]:
import re
import string

def normalize_text(
    text_lines: list[str],
    allowed_chars: str = "+-*/^%",
    allow_punctuation: bool = False,
    lang: str = "all",
    keep_case: bool = False,
    preserve_spaces: bool = False
) -> list[str]:
    """
    Нормализует список строк:
      1. Оставляет только разрешённые символы (по режиму lang + allowed_chars).
      2. Убирает пустые строки, табы, пробелы по краям.
         Если preserve_spaces=False — схлопывает множественные пробелы.
      3. По умолчанию переводит всё в нижний регистр (если keep_case=False).

    :param text_lines: список строк для обработки
    :param allowed_chars: строка с дополнительными разрешёнными символами
    :param allow_punctuation: если True — разрешаются все стандартные знаки препинания
    :param lang: 
        "all"         — буквы любого языка + цифры (по умолчанию)
        "ru"          — только кириллица + цифры
        "en"          — только латиница + цифры
        "digits_only" — только цифры
    :param keep_case: если True — сохраняется исходный регистр букв
    :param preserve_spaces: если True — не схлопывать множественные пробелы
    :return: список нормализованных строк
    """
    # базовый набор разрешённых символов
    extra = allowed_chars
    if allow_punctuation:
        extra += string.punctuation
    escaped_allowed = re.escape(extra)

    # выбор паттерна по lang
    if lang == "ru":
        letters = "А-Яа-яЁё"
        pattern = rf"[^0-9{letters}\s{escaped_allowed}]"
    elif lang == "en":
        letters = "A-Za-z"
        pattern = rf"[^0-9{letters}\s{escaped_allowed}]"
    elif lang == "digits_only":
        pattern = rf"[^0-9\s{escaped_allowed}]"
    else:  # all
        pattern = rf"[^\w\s{escaped_allowed}]"

    normalized = []
    for line in text_lines:
        # 1. фильтрация символов
        line = re.sub(pattern, "", line, flags=re.UNICODE)

        # 2. пробелы/табы
        line = line.replace("\t", " ")
        if not preserve_spaces:
            line = " ".join(line.split())
        else:
            line = line.strip()

        # 3. регистр
        if not keep_case:
            line = line.lower()

        # убираем пустые строки
        if line:
            normalized.append(line)

    return normalized

normalize_text(["вкео,лл hhjk5?6 $hf+f_вв"])


In [ ]:
# убирает ударения     ◌́ (U+0301 COMBINING ACUTE ACCENT)   !!! 
import unicodedata
text = unicodedata.normalize("NFC", text)        # что ещё может делать unicodedata
#  или проверять через \u0301


In [ ]:
# форма списка в форму словаря  ????  исправить - "+" надо перед ударной гласной
import unicodedata

def convert_accented_words(text: str) -> dict[str, str]:
    result = {}
    lines = text.strip().splitlines()

    for line in lines:
        word = line.strip().lstrip("0123456789. ")
        if not word:
            continue

        # Преобразуем в разложенную форму (NFD), чтобы акут стал отдельным символом
        decomposed = unicodedata.normalize("NFD", word)
        chars = list(decomposed)

        # Найти индекс ударной гласной
        for i in range(len(chars) - 1):
            if unicodedata.name(chars[i], "").startswith("CYRILLIC") and \
               unicodedata.name(chars[i + 1], "").startswith("COMBINING ACUTE"):
                # Ударная гласная найдена
                base = ''.join(chars[:i + 1])  # включая гласную
                tail = ''.join(chars[i + 2:])  # после акута
                clean = unicodedata.normalize("NFC", base + tail)
                marked = unicodedata.normalize("NFC", base + "+" + tail)
                result[clean] = marked
                break
        else:
            # Если акут не найден — оставить как есть
            result[word] = word

    return result


text = """
1. вертолётострое́ние
2 вертолётострои́тель
3. виндсёрфингист
 вперёдсмотря́щий
гёрлфре́нд
"""

converted = convert_accented_words(text)
for k, v in converted.items(): 
    print(f'"{k}": "{v}",')

ещё 

In [ ]:
# PUNCTUATION = r".,!?;:()«»′\"'…—–-"
# SPETIAL_SYMBOLS = r"/*+%#@¿¡№$€£¥°℃Å℉©®™§_^=<>⟨⟩[]{}|\\~`"  # ¢₹  ↑↓←→   так придётся добавить пункт "научные" знаки )  


#   AAAAAA!       https://ru.piliapp.com/symbol/
#             PUNCTUATION = ".,!?;:()«»—–-…"   # препинаки   !!!!  
# больше      PUNCTUATION = ".,!?;:()«»\"'—–-…/*+%"      .,!?;:\-–()«»′“‘\"'…
# ещё больше  PUNCTUATION = r".,!?;:()«»\"'—–-…/*+%#@¿"
#             PUNCTUATION = r"—–-/*+%#@¿¡№$€£¥°℃Å℉©®™§%^&*_=<>⟨⟩[]{}|\\~`"

регулярки

In [ ]:
import re


regex_rem_num_lat = re.compile(r"\d+|[a-zA-Z]+", flags=re.UNICODE)
print(regex_rem_num_lat.findall("abc 123 деф 45x"))

In [ ]:
l = [1,2,3,4,5]
m = [7,3,4,5]
diff = len(l) - len(m)
l = l[diff:]
print(diff)
print(l)